# Introduction to Machine Learning, UZH 2018, Group Project
### Group 2: Barbara Capl, Mathias Lüthi, Pamela Matias, Stefanie Rentsch
##       
# 2. Feature Selection 
# A. with RandomForestClassifier (RF)

###        
In this section we use the feature matrices and the response vector that we prepared in chapter 1.  

#### We use two different versions (created in chapter 1):
Version 1: Feature Matrix consists only of the Ratios                                                                        
Version 2: Feature Matrix consists of Ratios + dummy variables for seasonality + other market data
####  We will do Feature Selection and generate feature matrices with only selected n features for each Version above


In [1]:
# hide unnecessary warnings ("depreciation" of packages etc.)
import warnings
warnings.filterwarnings('ignore')

# import packages
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import Imputer
from sklearn.ensemble import RandomForestClassifier

## 2.0. SETTINGS: choose which Feature Matrix (Version 1 or 2) you want to use

#### Choose the Dataset Version you want
VERSION = 1; Feature Matrix with only ratios                                  
VERSION = 2;  Feature Matrix with ratios + saisonality + other market data


In [2]:
############### CHOOSE
VERSION = 1
"""
INSERT NUMBER 1 or 2    
"""



# Defining sel_state variable for usage later and printing text according to your choise   
if VERSION == 1:
    sel_version = 'based on Dataset with only the Ratios Dataset as predicive Features.'
    print('You chose VERSION 1 as working dataset. The following Feature Selection will be therefore ' + sel_version)
elif VERSION == 2:
    sel_version = 'based on Dataset with Ratios + Seasonality + other Market Data as predictive Features.'
    print('You chose VERSION 2 as working dataset. The following Feature Selection will be therefore ' + sel_version)
else: raise ValueError('VERSION must be either 1 or 2')

You chose VERSION 1 as working dataset. The following Feature Selection will be therefore based on Dataset with only the Ratios Dataset as predicive Features.


## 2.1. Preparation

### 2.1.1 Import the Response Vector and the Feature Matrices

In [3]:
# Load in the response vector pre-generated in Chater 1 (Preparation of Data)
y = pd.read_csv('Data/generated_datasets/response_1.csv', sep=',', header=0)

In [4]:
# Load in te feature matrix according to the version you chose in section 2.0. above
if VERSION == 1:
    X = pd.read_csv('Data/generated_datasets/features_ratios_1.csv', sep=',', header=0)
elif VERSION == 2:
    X = pd.read_csv('Data/generated_datasets/features_additional_1.csv', sep=',', header=0)
else: raise ValueError('VERSION value must be either 1 or 2')

### 2.1.2. Train/ Test Split and fill missing values with median

Note: Standardization is not done here because it is not necessary for Random Forest.                                        

### Train/ Test Split

Apply on chosen unsplitted, unimputed and unstandardized feature matrix and response vector.

In [5]:
# Train/test split, into 20% test size and 80% train size because it is a relatively small dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

# Extract all the feature labels for the respective chosen version
feature_labels = list(X)

print('Chosen features:')
display(feature_labels)

Chosen features:


['CAPEI',
 'bm',
 'evm',
 'pe_op_basic',
 'pe_op_dil',
 'pe_exi',
 'pe_inc',
 'ps',
 'pcf',
 'dpr',
 'npm',
 'opmbd',
 'opmad',
 'gpm',
 'ptpm',
 'cfm',
 'roa',
 'roe',
 'roce',
 'efftax',
 'aftret_eq',
 'aftret_invcapx',
 'aftret_equity',
 'pretret_noa',
 'pretret_earnat',
 'GProf',
 'equity_invcap',
 'debt_invcap',
 'totdebt_invcap',
 'capital_ratio',
 'int_debt',
 'int_totdebt',
 'cash_lt',
 'invt_act',
 'rect_act',
 'debt_at',
 'debt_ebitda',
 'short_debt',
 'curr_debt',
 'lt_debt',
 'profit_lct',
 'ocf_lct',
 'cash_debt',
 'fcf_ocf',
 'lt_ppent',
 'dltt_be',
 'debt_assets',
 'debt_capital',
 'de_ratio',
 'intcov',
 'intcov_ratio',
 'cash_ratio',
 'quick_ratio',
 'curr_ratio',
 'cash_conversion',
 'inv_turn',
 'at_turn',
 'rect_turn',
 'pay_turn',
 'sale_invcap',
 'sale_equity',
 'sale_nwc',
 'rd_sale',
 'adv_sale',
 'staff_sale',
 'accrual',
 'ptb',
 'PEG_trailing',
 'divyield',
 'PEG_1yrforward',
 'PEG_ltgforward']

### Filling missing values (NaN)

We use Imputer to fill missing entries with median only AFTER the split, to circumvent the bias that both splits would include information of each other.

In [6]:
# Use a median fill for train set
imp = Imputer(missing_values=np.nan, strategy = 'median' , axis=0)
imputed_dataset = pd.DataFrame(imp.fit_transform(X_train))
imputed_dataset.columns = X_train.columns
imputed_dataset.index = X_train.index
X_train = imputed_dataset

# Use a median fill for the test set
imputed_dataset = pd.DataFrame(imp.fit_transform(X_test))
imputed_dataset.columns = X_test.columns
imputed_dataset.index = X_test.index
X_test = imputed_dataset

### 2.2.3. Print out Shape and Form of Feature Matrix and Response Vector

### Train set

In [7]:
# Check if there is the approximately same percentage of '1' the train response vector
print('Ratio of "UP" (Train) = ' + str(round(y_train['0'].sum() / y_train.size, 4)*100) + '%', '\n')

# Print and display properties and head of the training set we will use in the following
print('Shape of Feature Matrix X_train = ' + str(X_train.shape), '\n')
print('Feature Matrix X_train: (below)')
display(X_train.head(3))
print('\n')
print('Response Vector y_train, Shape = '+str(y_train.shape))
display(y_train.head(3))

Ratio of "UP" (Train) = 58.07% 

Shape of Feature Matrix X_train = (2836, 71) 

Feature Matrix X_train: (below)


,CAPEI,bm,evm,pe_op_basic,pe_op_dil,pe_exi,pe_inc,ps,pcf,dpr,...,sale_nwc,rd_sale,adv_sale,staff_sale,accrual,ptb,PEG_trailing,divyield,PEG_1yrforward,PEG_ltgforward
1530,20.773,0.547,10.644,15.468,15.633,18.484,18.484,3.703,9.957,0.623,...,1.4240,0.136,0.058,0.000,0.048,1.811,1.181,0.0354,13.678,5.963
1397,25.089,0.461,9.246,17.203,17.296,39.232,39.232,0.689,8.509,2.175,...,4.9910,0.031,0.000,0.000,0.047,2.189,0.668,0.0398,1.521,5.702
2238,12.337,0.911,12.474,9.135,9.209,13.062,13.062,2.034,1.993,0.337,...,5.8105,0.000,0.024,0.291,0.038,1.076,3.048,0.0268,0.380,2.248




Response Vector y_train, Shape = (2836, 1)


,0
1530,0
1397,1
2238,0


### Test set

In [8]:
# Check if there is the approximately same percentage of '1' in the test response vector
print('Ratio of "UP" (Test)  =  ' + str(round(y_test['0'].sum() / y_test.size, 4)*100) + '%', '\n')

# Print and display properties and head of the training set we will use in the following
print('Shape of Feature Matrix X_test = ' + str(X_test.shape), '\n')
print('Feature Matrix X_test')
display(X_test.head(3))
print('\n')
print('Response Vector y_test, Shape = '+str(y_test.shape))
display(y_test.head(3))

Ratio of "UP" (Test)  =  57.04% 

Shape of Feature Matrix X_test = (710, 71) 

Feature Matrix X_test


,CAPEI,bm,evm,pe_op_basic,pe_op_dil,pe_exi,pe_inc,ps,pcf,dpr,...,sale_nwc,rd_sale,adv_sale,staff_sale,accrual,ptb,PEG_trailing,divyield,PEG_1yrforward,PEG_ltgforward
817,20.983,0.254,8.585,16.127,16.224,16.224,16.224,3.318,10.484,0.287,...,6.768,0.031,0.006,0.000,0.084,4.189,1.022,0.0187,1.414,1.330
2592,21.663,0.235,13.619,16.311,16.489,18.555,18.555,2.379,7.709,0.199,...,7.169,0.000,0.000,0.195,0.039,4.281,1.147,0.0122,1.737,1.572
1475,16.327,0.513,6.716,9.520,9.520,15.681,15.550,3.293,8.490,0.792,...,1.286,0.162,0.054,0.000,0.077,2.198,1.022,0.0343,-0.973,6.149




Response Vector y_test, Shape = (710, 1)


,0
817,1
2592,0
1475,1


###    
## 2.2. Selecting & Extracting Features

### 2.2.1.  Fit forest and show list of best 'n' features according to explanatory value

In [58]:
# rs = first random state we choose, 
# n = number of best features we want to select out, 
# m = number of random states we want to have tried out
rs = 0
n = 20
m = 50

# define new empty lists
feature_importances = [0] * 71

# Loop until random_state = rs = 10
while rs < m:
    forest = RandomForestClassifier(random_state = rs)
    forest.max_depth = 6
    forest.fit(X_train, y_train)
    
    # Apply forest fature importance to get an array of feature importances for each feature
    feature_importances = forest.feature_importances_ + feature_importances
        
    rs += 1

# sort features in line with their importance for the prediction (therefore ::-1)
indices = np.argsort(feature_importances)[::-1]

selected_labels = [feature_labels[i] for i in indices[:n]]
selected_importances = sorted(feature_importances, reverse=True)[:n]

important_features = pd.DataFrame({'labels':selected_labels, 'importances':selected_importances},
                                  columns=['labels', 'importances'])

print('Most important features after ' + str(rs) + ' random states')
display(important_features)

del rs, m

Most important features after 50 random states


,labels,importances
0,pe_op_dil,2.329614
1,CAPEI,2.267174
2,pe_op_basic,2.193411
3,divyield,1.863889
4,pe_exi,1.816294
5,pcf,1.747672
6,ptb,1.688619
7,PEG_ltgforward,1.662415
8,bm,1.541856
9,evm,1.541651


### 2.2.2.  Plotting results

In [68]:
# Plot Feature Importance (both cumulative and individual)

plt.figure()
important_features['importances'].plot(kind='bar')

### 2.2.3. Extract best 'n' Variables and save into new Feature Matrices


### Extract

Automatically etract the chosen number n best variables directly from the feature matrices X_train and X_test into new ones X_train_f and X_test_f.                                                                                                                
The f in the end stands for feature selection with random'f'orest.


In [ ]:
################################## NEW
##################################

# Extract the overall n best features INCLUDING the weighted returns
# remember, we set n = 15 in the beginning for the randomforest in section 2.2.1.
final_selection1 = best_features_cumsum.Feature_Label[0:n]
print('Selected Features including weighted returns:')
display(final_selection1)
print("")
# save as list for further use
final_selection1 = list(final_selection1)


# Extract the overall n best features EXCLUDING the weighted returns
m = n + 4
final_selection2 = best_features_cumsum.Feature_Label[4:m]
final_selection2.reset_index(drop = True, inplace = True)
print('Selected Features without weighted returns:')
display(final_selection2)
# save as list for further use
final_selection2 = list(final_selection2)

In [ ]:
################################## NEW
##################################

# features including volume weighted returns
best_features1 = final_selection1
# feaures excluding volume weighted returns
best_features2 = final_selection2

# extract features from feature matrix
X_train_f1 = X_train[best_features1]
X_train_f = X_train[best_features2]
X_test_f1 = X_test[best_features1]
X_test_f = X_test[best_features2]

#feature_labels_f = X_train_f.columns[:]

In [ ]:
#### OLD

# generate new empty list for best features to be extracted into
#best_features = []

# loop to append best n features onto the new list best_features
#n = 15
#for i in range(n):
 #   best_features.append(feature_labels[indices[i]])
#del i

# extract data of train and test feature matrix where column names are the same as those of the n best features
# The f at the end of the datasets refer to the fact that the features were selected with random'f'orest
#X_train_f = X_train[best_features]
#X_test_f = X_test[best_features]
#feature_labels_f = X_train_f.columns[:]

# display and print results
#print('Best Features after Random Forest are: ', '\n')
#display(np.array(best_features))
#print('Head of X_train_f is: (below) ', '\n')
#display(X_train_f.head(2))
#print('\n')
#print('Extracted feature labels are: (below)')
#display(feature_labels_f[:])

In [ ]:
################################## NEW
##################################

# Show feature matrices (train, test) after feature selection
print("")
print('Feature Matrix with selected Features (Train):')
display(X_train_f1.head(2))
display(X_train_f1.shape)
display(X_train_f.head(2))
display(X_train_f.shape)


print("")
print('Feature Matrix with selected Features (Test):')
display(X_test_f1.head(2))
display(X_test_f1.shape)
display(X_test_f.head(2))
display(X_train_f.shape)

### Save

Feature Matrices and Response Vectors after Train/Test Split and after Feature Selection are saved as csv:             
Version 1 (ratios as explanatory variables)                                                                          
Version 2 (ratios + seasonality + market data).

In [ ]:
# Save generated splitted datasets (feature matrices and response vectors) after having relevant feature selected
# The 'f' at the end of the name of the dataset stands for the fact that features have been selected with random'f'orest
if VERSION == 1:
    X_train_f1.to_csv('Data/generated_splits/features_selected_randomforest/X1_train_f.csv', index = False)
    X_test_f1.to_csv('Data/generated_splits/features_selected_randomforest/X1_test_f.csv', index = False)
    y_train.to_csv('Data/generated_splits/features_selected_randomforest/y1_train_f.csv', index = False)
    y_test.to_csv('Data/generated_splits/features_selected_randomforest/y1_test_f.csv', index = False)
elif VERSION == 2:
    X_train_f1.to_csv('Data/generated_splits/features_selected_randomforest/X2_train_f.csv', index = False)
    X_test_f1.to_csv('Data/generated_splits/features_selected_randomforest/X2_test_f.csv', index = False)
    y_train.to_csv('Data/generated_splits/features_selected_randomforest/y2_train_f.csv', index = False)
    y_test.to_csv('Data/generated_splits/features_selected_randomforest/y2_test_f.csv', index = False)
else: raise ValueError('VERSION value must be either 1 or 2')

In [ ]:
display(X_train_f1.shape)
display(X_test_f1.shape)